# Imports and set up

In [17]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [18]:
aapl_df = pd.read_csv("../data/aapl.csv", parse_dates=["Date"], index_col=["Date"]) # import the data to a df, sets the Date column as DateTime and also sets it as the index

In [19]:
pd.set_option('display.float_format', '{:,.3f}'.format) #sets some global number formats which should make the data easier to read.

# Addind Date Columns

In [20]:
aapl_df["Year"] = aapl_df.index.year
aapl_df["Month"] = aapl_df.index.month
aapl_df["Day"] = aapl_df.index.day

# this adds a year, month and day column to the data. 

In [21]:
aapl_df.tail()

,Close,High,Low,Open,Volume,Year,Month,Day
Date,,,,,,,,
2025-08-07,219.781,220.600,216.334,218.632,90224800,2025,8,7
2025-08-08,229.090,230.738,219.001,220.580,113854000,2025,8,8
2025-08-11,227.180,229.560,224.760,227.920,61806100,2025,8,11
2025-08-12,229.650,230.800,227.070,228.010,55626200,2025,8,12
2025-08-13,233.330,235.000,230.430,231.070,69833200,2025,8,13


# Initial EDA

## Basics

In [22]:
aapl_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11258 entries, 1980-12-12 to 2025-08-13
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   11258 non-null  float64
 1   High    11258 non-null  float64
 2   Low     11258 non-null  float64
 3   Open    11258 non-null  float64
 4   Volume  11258 non-null  int64  
 5   Year    11258 non-null  int32  
 6   Month   11258 non-null  int32  
 7   Day     11258 non-null  int32  
dtypes: float64(4), int32(3), int64(1)
memory usage: 659.6 KB


In [23]:
print(aapl_df.head())
print(aapl_df.tail())

            Close  High   Low  Open     Volume  Year  Month  Day
Date                                                            
1980-12-12  0.098 0.099 0.098 0.098  469033600  1980     12   12
1980-12-15  0.093 0.094 0.093 0.094  175884800  1980     12   15
1980-12-16  0.086 0.087 0.086 0.087  105728000  1980     12   16
1980-12-17  0.089 0.089 0.089 0.089   86441600  1980     12   17
1980-12-18  0.091 0.092 0.091 0.091   73449600  1980     12   18
             Close    High     Low    Open     Volume  Year  Month  Day
Date                                                                   
2025-08-07 219.781 220.600 216.334 218.632   90224800  2025      8    7
2025-08-08 229.090 230.738 219.001 220.580  113854000  2025      8    8
2025-08-11 227.180 229.560 224.760 227.920   61806100  2025      8   11
2025-08-12 229.650 230.800 227.070 228.010   55626200  2025      8   12
2025-08-13 233.330 235.000 230.430 231.070   69833200  2025      8   13


In [24]:
aapl_df.isna().sum()

Close     0
High      0
Low       0
Open      0
Volume    0
Year      0
Month     0
Day       0
dtype: int64

In [25]:
aapl_df.describe()

,Close,High,Low,Open,Volume,Year,Month,Day
count,"11,258.000","11,258.000","11,258.000","11,258.000","11,258.000","11,258.000","11,258.000","11,258.000"
mean,26.009,26.273,25.720,25.988,"311,985,889.092","2,002.754",6.514,15.738
std,53.997,54.534,53.399,53.940,"333,943,488.822",12.898,3.430,8.748
min,0.038,0.038,0.038,0.038,0.000,"1,980.000",1.000,1.000
25%,0.245,0.250,0.239,0.245,"108,355,900.000","1,992.000",4.000,8.000
50%,0.462,0.469,0.455,0.462,"200,550,000.000","2,003.000",7.000,16.000
75%,20.341,20.533,20.216,20.377,"391,638,800.000","2,014.000",9.000,23.000
max,258.104,259.180,256.719,257.277,"7,421,640,800.000","2,025.000",12.000,31.000


## Find min and max's

In [26]:
aapl_df.loc[aapl_df["Close"].idxmin()] # this returns the full row of the min Close value.

Close              0.038
High               0.038
Low                0.038
Open               0.038
Volume   164,326,400.000
Year           1,982.000
Month              7.000
Day                8.000
Name: 1982-07-08 00:00:00, dtype: float64

In [27]:
# if just want the min value can use.
aapl_df["Close"].min()

np.float64(0.0376811027526855)

In [28]:
aapl_df.loc[aapl_df["Close"].idxmax()] # same for max

Close           258.104
High            259.180
Low             256.719
Open            257.277
Volume   27,237,100.000
Year          2,024.000
Month            12.000
Day              26.000
Name: 2024-12-26 00:00:00, dtype: float64

## Visuals

In [37]:
fig = px.line(aapl_df["Close"], title="Apple Closing Stock")
fig.show()

In [41]:
px.bar(aapl_df["Volume"], 
       title="Apple Stock Info",
       color_discrete_sequence=["red"] )

# Grouping and Filtering

### Grouping

In [47]:
aapl_df.groupby(["Year"]).mean().drop(columns=["Month", "Day"])

,Close,High,Low,Open,Volume
Year,,,,,
1980,0.104,0.105,0.104,0.104,"103,450,092.308"
1981,0.083,0.084,0.083,0.084,"32,398,988.142"
1982,0.066,0.067,0.065,0.066,"84,446,671.937"
1983,0.128,0.131,0.125,0.128,"175,940,755.731"
1984,0.092,0.094,0.090,0.092,"165,925,033.992"
1985,0.069,0.071,0.069,0.070,"180,524,888.889"
1986,0.111,0.113,0.109,0.111,"210,763,724.901"
1987,0.267,0.274,0.261,0.267,"236,250,232.411"
1988,0.287,0.292,0.283,0.287,"163,213,350.198"


In [48]:
aapl_df.groupby(["Month"]).mean().drop(columns=["Year", "Day"])

,Close,High,Low,Open,Volume
Month,,,,,
1,25.219,25.506,24.918,25.220,"417,861,654.045"
2,26.286,26.530,25.970,26.233,"319,544,756.199"
3,25.342,25.646,25.029,25.325,"308,070,243.452"
4,25.538,25.841,25.225,25.503,"321,290,043.656"
5,26.116,26.373,25.842,26.110,"284,584,379.180"
6,25.976,26.236,25.718,25.954,"282,359,396.559"
7,28.477,28.717,28.191,28.435,"289,798,425.525"
8,27.111,27.373,26.818,27.069,"271,007,441.667"
9,25.101,25.412,24.822,25.163,"313,134,343.289"


In [54]:
aapl_df.groupby(["Year", "Month"]).mean().drop(columns=["Day"])

Close    High     Low    Open          Volume
Year Month                                                
1980 12      0.104   0.105   0.104   0.104 103,450,092.308
1981 1       0.108   0.109   0.108   0.109  28,999,466.667
     2       0.090   0.091   0.090   0.091  16,927,326.316
     3       0.085   0.086   0.085   0.085  31,850,763.636
     4       0.093   0.094   0.093   0.093  25,568,000.000
...            ...     ...     ...     ...             ...
2025 4     200.693 204.781 195.771 199.278  76,538,980.952
     5     203.628 205.637 201.265 203.799  56,939,438.095
     6     200.430 202.595 198.756 200.673  55,020,465.000
     7     211.041 212.952 209.617 211.099  49,087,981.818
     8     217.778 220.880 214.179 216.632  80,391,811.111

[537 rows x 5 columns]

In [69]:
aapl_df.groupby([pd.Grouper(freq="ME")]).mean().drop(columns=["Year", "Month", "Day"])

,Close,High,Low,Open,Volume
Date,,,,,
1980-12-31,0.104,0.105,0.104,0.104,"103,450,092.308"
1981-01-31,0.108,0.109,0.108,0.109,"28,999,466.667"
1981-02-28,0.090,0.091,0.090,0.091,"16,927,326.316"
1981-03-31,0.085,0.086,0.085,0.085,"31,850,763.636"
1981-04-30,0.093,0.094,0.093,0.093,"25,568,000.000"
...,...,...,...,...,...
2025-04-30,200.693,204.781,195.771,199.278,"76,538,980.952"
2025-05-31,203.628,205.637,201.265,203.799,"56,939,438.095"
2025-06-30,200.430,202.595,198.756,200.673,"55,020,465.000"


In [76]:
monthly_df = aapl_df.resample("ME").mean().drop(columns=["Year", "Month", "Day"])

In [77]:
px.line(monthly_df)

In [124]:
daily_df = (aapl_df["Close"]
            .groupby([aapl_df.index.day_of_week,aapl_df.index.month])
            .mean()
            )
daily_df

Date  Date
0     1      23.331
      2      25.928
      3      25.336
      4      25.299
      5      26.270
      6      25.235
      7      28.835
      8      27.033
      9      25.142
      10     25.085
      11     25.388
      12     25.612
1     1      26.208
      2      26.222
      3      25.035
      4      26.189
      5      25.854
      6      26.236
      7      28.343
      8      27.569
      9      24.520
      10     25.685
      11     25.688
      12     26.969
2     1      25.594
      2      26.106
      3      25.637
      4      25.740
      5      26.039
      6      25.656
      7      29.418
      8      27.302
      9      25.031
      10     24.739
      11     25.955
      12     24.756
3     1      24.810
      2      26.978
      3      25.193
      4      25.055
      5      26.133
      6      26.075
      7      27.814
      8      26.798
      9      25.183
      10     24.976
      11     25.523
      12     27.038
4     1      25.815
      2  

In [151]:
daily_df = (aapl_df
            .groupby(["Day", "Month"])
            .mean()
            .reset_index()
            )
daily_df

,Day,Month,Close,High,Low,Open,Volume,Year
0,1,2,25.599,25.740,25.161,25.426,"318,110,746.875","2,003.250"
1,1,3,25.069,25.326,24.708,25.029,"326,373,400.000","2,003.125"
2,1,4,29.976,30.190,29.637,29.966,"239,961,906.667","2,004.267"
3,1,5,26.233,26.556,25.940,26.027,"314,528,665.625","2,003.344"
4,1,6,22.093,22.266,21.875,22.039,"304,676,059.375","2,002.250"
...,...,...,...,...,...,...,...,...
358,31,5,27.271,27.510,27.047,27.277,"307,785,800.000","2,003.500"
359,31,7,29.895,30.160,29.589,29.846,"272,057,612.500","2,003.344"
360,31,8,25.426,25.702,25.257,25.486,"275,006,481.250","2,002.250"
361,31,10,26.097,26.303,25.787,26.116,"323,872,880.645","2,003.194"


In [153]:
matrix = daily_df.pivot(index="Day", columns="Month", values="Close")
matrix

Month,1,2,3,4,5,6,7,8,9,10,11,12
Day,,,,,,,,,,,,
1,NaN,25.599,25.069,29.976,26.233,22.093,28.677,31.549,28.535,22.387,29.270,24.972
2,26.969,24.837,21.063,21.624,28.787,27.282,26.660,29.093,21.529,24.262,24.112,26.881
3,30.345,26.919,27.220,25.150,26.316,27.241,32.620,24.795,24.930,25.453,23.362,23.962
4,24.812,23.730,27.652,28.552,21.883,24.719,NaN,29.387,21.654,28.611,25.286,25.879
5,23.713,25.134,24.086,27.701,26.847,27.441,28.981,30.018,26.011,23.985,22.914,26.528
6,26.251,26.357,25.914,22.455,27.175,29.503,23.107,27.342,29.932,23.047,24.902,30.586
7,23.543,30.955,28.766,21.725,24.105,26.583,28.560,29.597,25.974,24.609,26.052,25.348
8,25.159,25.949,24.508,26.017,26.578,22.257,29.633,31.936,23.761,22.290,29.381,25.316
9,18.656,25.074,20.745,25.783,28.925,27.167,27.212,28.758,25.375,24.697,24.237,27.289


In [156]:
px.scatter(matrix)

## Filtering

In [91]:
monthly_df.loc[
    (monthly_df.index.year > 2020)
]

,Close,High,Low,Open,Volume
Date,,,,,
2021-01-31,129.631,131.612,127.625,129.693,"117,908,526.316"
2021-02-28,128.314,129.812,126.751,128.793,"96,518,715.789"
2021-03-31,118.928,120.490,117.497,119.139,"115,235,573.913"
2021-04-30,128.630,129.702,127.315,128.493,"89,993,214.286"
2021-05-31,123.890,125.232,122.952,124.135,"85,596,745.000"
2021-06-30,127.036,127.713,125.865,126.577,"73,026,818.182"
2021-07-31,141.875,142.963,140.310,141.319,"91,382,623.810"
2021-08-31,145.023,146.049,143.739,144.703,"66,433,763.636"
2021-09-30,145.188,146.841,144.125,145.869,"85,611,195.238"


In [92]:
monthly_df.loc[
    (monthly_df.index.year > 2015) & (monthly_df["Close"] > 200)
]

,Close,High,Low,Open,Volume
Date,,,,,
2024-06-30,205.070,207.827,202.741,204.803,"90,418,531.579"
2024-07-31,223.300,225.360,220.746,223.299,"52,413,627.273"
2024-08-31,220.533,222.603,217.651,219.655,"51,030,318.182"
2024-09-30,222.721,224.544,220.431,222.693,"61,604,885.000"
2024-10-31,228.990,230.585,227.033,229.090,"40,466,782.609"
2024-11-30,226.943,228.201,224.767,225.860,"44,579,415.000"
2024-12-31,248.441,249.924,246.001,247.453,"46,567,433.333"
2025-01-31,233.757,236.733,231.018,234.271,"60,008,820.000"
2025-02-28,237.865,239.932,234.681,236.728,"45,382,752.632"


In [93]:
monthly_df.loc[
    (monthly_df.index.year > 2015) & (monthly_df["Close"] > 200), ["Close"]
]

,Close
Date,
2024-06-30,205.070
2024-07-31,223.300
2024-08-31,220.533
2024-09-30,222.721
2024-10-31,228.990
2024-11-30,226.943
2024-12-31,248.441
2025-01-31,233.757
2025-02-28,237.865


## Combining

In [159]:
monthly_lows = aapl_df["Close"].resample("ME").min()
monthly_lows

Date
1980-12-31     0.086
1981-01-31     0.097
1981-02-28     0.081
1981-03-31     0.074
1981-04-30     0.083
               ...  
2025-04-30   171.999
2025-05-31   195.049
2025-06-30   195.418
2025-07-31   207.335
2025-08-31   202.151
Freq: ME, Name: Close, Length: 537, dtype: float64

In [170]:
recent_months = (aapl_df
                 .loc[aapl_df.index.year > 2023, ["Close"]]
                 .resample("ME")
                 .mean())
recent_months

,Close
Date,
2024-01-31,186.148
2024-02-29,183.387
2024-03-31,171.465
2024-04-30,168.395
2024-05-31,185.132
2024-06-30,205.070
2024-07-31,223.300
2024-08-31,220.533
2024-09-30,222.721
